In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
import openai
import getpass
# Prompt the user for the API key
api_key = getpass.getpass("Enter your OpenAI API key: ")
# Assign the API key to the OpenAI client
openai.api_key = api_key

Enter your OpenAI API key: ··········


In [ ]:
from openai import OpenAI


In [ ]:
client = OpenAI(
    api_key=api_key,
)

def generate(prompt):
    try:
        openai.api_key = api_key
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {
                    "role": "system",
                    "content": "You are an intelligent meal planner assistant. Your primary task is to gather information about the user (including weight, height, goal) and to understand user queries related to meal planning and provide helpful responses. This includes recognizing the user's intent (e.g., seeking a recipe, needing a shopping list, asking for nutritional advice) and identifying specific details (slots) such as dietary preferences, meal types, preferred cuisines, and ingredients. You should also consider users' dietary restrictions and nutritional goals when providing recommendations."
                },
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
        )
        return completion.choices[0].message.content
    except Exception as e:
        return str(e)

In [20]:
import re

class MealPlannerAssistant:
    def __init__(self):
        self.user_profile = {
            "height": None,  # in cm
            "weight": None,  # in kg
            "gender": None,
            "dietary_restriction": None,  # Categorized by predefined list
            "goal": None
        }
        self.questions = {
            "height": "Could you tell me your height? (e.g., 170cm or 5'7\")",
            "weight": "What's your weight? (e.g., 70kg or 155lbs)",
            "gender": "What's your gender?",
            "dietary_restriction": "Do you have any dietary restrictions? (e.g., vegan, keto)",
            "goal": "What's your goal? (e.g., weight loss, maintenance, muscle gain)"
        }

    def ask_questions(self):
        for key, question in self.questions.items():
            valid_response = False
            while not valid_response:
                if self.user_profile[key] is None:
                    print(question)
                    response = input().strip()
                    valid_response, processed_response = self.validate_and_process_response(key, response)
                    if valid_response:
                        self.user_profile[key] = processed_response
                    else:
                        print(f"Invalid response for {key}. Please try again.")

    def validate_and_process_response(self, topic, response):
        if topic == "dietary_restriction":
            prompt = f"From this response '{response}', can we categorize the user's dietary restriction into one of the following: None, Keto, Vegan, Vegetarian, Gluten-Free, Lactose-Intolerant, Paleo, Halal, Other? Please select and list out the closest option."
        if topic in ["height", "weight"]:
            prompt = f"From this response '{response}', can we tell what the user's {topic} is and the unit? Please answer Yes or No, and if Yes, provide the user's {topic}."
        else:
            prompt = f"From this response '{response}', can we tell what the user's {topic} is? Please answer Yes or No, and if Yes, provide the user's {topic}."

        ai_response = generate(prompt)
        print(ai_response)

        if "no" in ai_response.lower().split() and topic != "dietary_restriction":
            return False, "Invalid response"
        else:
            processed_response = self.process_ai_response(topic, ai_response)
            return True, processed_response

    def process_ai_response(self, topic, response):
        if topic in ["height", "weight"]:
            prompt = f"Convert the user's {topic} '{response}' into standard units (write cm for height, kg for weight)."
        elif topic == "gender":
            prompt = f"Based on this response '{response}', how can we best classify the user's gender? Consider traditional categories like 'male' or 'female', but also note if the response suggests a non-binary or prefer-not-to-say option."
        elif topic == "goal":
            prompt = f"Given this response '{response}', what is the user's fitness or health goal? Classify it into one of the following categories and list it: weight loss, maintenance, muscle gain, or specify 'other' if none of these categories accurately capture the user's goal."
        else:
            return self.finalize_response(topic, response)
        ai_interpreted_response = generate(prompt)

        # Apply NLP techniques for final standardization
        return self.finalize_response(topic, ai_interpreted_response)

    def finalize_response(self, topic, response):
        #return self._interpret_goal(response)
        if topic == "height":
            return self._convert_height_to_cm(response)
        elif topic == "weight":
            return self._convert_weight_to_kg(response)
        elif topic == "gender":
            return self._interpret_gender(response)
        elif topic == "goal":
            return self._interpret_goal(response)
        else:
          return response

    def _interpret_gender(self, response):
        return next((value for word in response.lower().split() for key, value in {
            "female": "Female",
            "male": "Male",
            "non-binary": "Non-binary",
            "prefer not to say": "Prefer not to specify",
        }.items() if key in word), "Prefer not to specify")


    def _interpret_goal(self, response):
        return next((value for word in response.split() for key, value in {
            "weight loss": "Weight Loss",
            "lose weight": "Weight Loss",
            "maintenance": "Maintenance",
            "muscle gain": "Muscle Gain",
        }.items() if key in response.lower()), "Other")


    def _convert_height_to_cm(self, response):
        # Handling centimeters directly or inches, e.g., 170cm or 67 inches
        cm_inches_pattern = re.compile(r"(\d+)\s*(cm|inch(es)?)?", re.IGNORECASE)
        match = cm_inches_pattern.search(response)
        if match:
            value, unit = int(match.group(1)), match.group(2)
            if not unit or "cm" in unit.lower():
                return f"{value}cm"
            elif "inch" in unit.lower():
                return f"{value * 2.54:.0f}cm"

        return "Invalid height format"

    def _convert_weight_to_kg(self, response):
        # Handling pounds and kilograms
        lbs_kg_pattern = re.compile(r"(\d+)\s*(kg|lb(s)?)?", re.IGNORECASE)
        match = lbs_kg_pattern.search(response)
        if match:
            value, unit = int(match.group(1)), match.group(2)
            if not unit or "kg" in unit.lower():
                return f"{value}kg"
            elif "lb" in unit.lower():
                return f"{value / 2.20462:.0f}kg"

        return "Invalid weight format"


    def is_complete(self):
        return all(value is not None for value in self.user_profile.values())

    def run(self):
        print("Welcome to the Meal Planner Assistant. I'll ask you a few questions to understand your preferences and needs.")
        while not self.is_complete():
            self.ask_questions()
        print("Thank you for providing the information. Here's your profile:")
        for key, value in self.user_profile.items():
            print(f"{key.replace('_', ' ').title()}: {value}")

if __name__ == "__main__":
    assistant = MealPlannerAssistant()
    assistant.run()


Welcome to the Meal Planner Assistant. I'll ask you a few questions to understand your preferences and needs.
Could you tell me your height? (e.g., 170cm or 5'7")
160 centimeters
Yes, the user's height is 160 centimeters.
What's your weight? (e.g., 70kg or 155lbs)
12
No
Invalid response for weight. Please try again.
What's your weight? (e.g., 70kg or 155lbs)
112 pounds
Yes, the user's weight is 112 pounds.
What's your gender?
female
Yes, the user's gender is Female.
Do you have any dietary restrictions? (e.g., vegan, keto)
nope
No
What's your goal? (e.g., weight loss, maintenance, muscle gain)
lose weight
Yes, the user's goal is to lose weight.
Thank you for providing the information. Here's your profile:
Height: 160cm
Weight: 50kg
Gender: Female
Dietary Restriction: No
Goal: Weight Loss


In [ ]:
# scraped recipes

In [ ]:
# Fine tune spacy NER for food recognition? Grocery list, food recommendation(ask the user for their pantry, foods they like, ect...)

What we need to do:
1. Incorporate a calorie calculator that calculates the user's ideal calorie intake
**2. Prompt the user what they look for in a recipe and their taste profile (i.e. cheap, easy, fast, taste, ....). We will write another validator for the user's response to this as well, and then ask the agent to summarize their preference for what they want in a recipe**
3. We will filter; first by dietary restrictions(no filter if none), then we can use the calorie amount (+ or - 100 calories is okay) to filter again, then finally use sentiment analysis(cosine similarity or BERT) to match what the agent summarized from part 2 to rank it
4. If no match, we will use prompting to ask the agent to generate a new recipe

Potentially: Use NER/toppic modeling to get their ingredients from their pantry list, and create a grocery list by cross-referencing the recipe's ingredient list, maybe for better ranking as well

In [1]:
# Need to implement NER/Topic modeling first, both on the user's preferences and the filtered recipes, ideally it would speed up the process
# Sample BERT model that ranks recipes based on the user's preferences


from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

class RecipeRanker:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased')
        self.model.eval()  # Set model to evaluation mode

    def encode(self, text):
        """
        Encodes a given text into embeddings using BERT.
        """
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = self.model(**inputs)
        # Use the pooler output (representing the entire sentence) for simplicity
        return outputs.pooler_output

    def rank_recipes_by_taste_profile(self, recipes, taste_profile):
        """
        Ranks recipes by their cosine similarity to the user's taste profile.
        """
        # Convert the user's taste profile into an embedding
        taste_profile_embedding = self.encode(taste_profile)

        # Calculate similarity scores and rank recipes
        ranked_recipes = []
        for recipe in recipes:
            recipe_embedding = self.encode(recipe['summary'])
            similarity_score = cosine_similarity(taste_profile_embedding, recipe_embedding)
            ranked_recipes.append((recipe, similarity_score.item()))

        # Sort recipes based on similarity score
        ranked_recipes.sort(key=lambda x: x[1], reverse=True)

        # Return the sorted list of recipes
        return [recipe for recipe, score in ranked_recipes]

# Example usage
if __name__ == "__main__":
    ranker = RecipeRanker()
    # Example recipes (might need to apply topic modeling)
    recipes = [
        {"name": "Healthy Avocado Toast", "summary": "Simple and healthy avocado toast that is perfect for a quick breakfast or lunch."},
        {"name": "Classic Italian Pasta", "summary": "Delicious Italian pasta with a rich tomato sauce and fresh herbs."},
        # Add more recipes as needed
    ]
    taste_profile = "Looking for healthy and quick breakfast options with avocado"

    ranked_recipes = ranker.rank_recipes_by_taste_profile(recipes, taste_profile)
    for recipe in ranked_recipes:
        print(recipe["name"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Classic Italian Pasta
Healthy Avocado Toast
